In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import metrics
from tensorflow.contrib import layers
from tensorflow.contrib import learn
from tensorflow.contrib.learn import *
from tensorflow.contrib import seq2seq

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

## Import helper functions for processing data
- [data_creator.py](/edit/data/data_creator.py)
- [utils.py](/edit/utils.py)

In [ ]:
from data.data_creator import *
from utils import *

## Define RNN model function for word segmentation

In [ ]:
def rnn_segment(features, targets, mode, params):
    seq_feature = features['seq_feature']
    seq_length = features['seq_length']
    
    # Create a variable matrix in which each row represents an embedding vector for a given character.
    with tf.variable_scope("emb"):
        embeddings = tf.get_variable("char_emb", shape=[params[???], params[???]])
    
    # Convert sequence features to sequence embeddings 
    seq_emb = tf.nn.embedding_lookup(params=???, ids=???)
    
    # Flatten each sequence embedding
    # [[1st char embedding], [2nd char embedding], [3rd char embedding]] => [flatten seq embedding]
    batch_size = tf.shape(seq_feature)[0]
    time_step = tf.shape(seq_feature)[1]
    flat_seq_emb = tf.reshape(seq_emb, shape=[batch_size, time_step, (params['k'] + 1) * params['emb_size']])
    
    cell = rnn.LSTMCell(num_units=???)
    if mode == ModeKeys.TRAIN:
        # Create a cell with added input and output dropout with given probabilities.
        cell = rnn.DropoutWrapper(cell, ???)
        
    # Project output from RNN cell into classes
    projection_cell = rnn.OutputProjectionWrapper(cell, ???)
    
    # Create a recurrent neural network from the projection_cell
    logits, _ = tf.nn.dynamic_rnn(???, dtype=tf.float32)
    
    # Training samples in each batch might have different lengths. 
    # We put them in the same matrix by padding each sample to have the same length as the longest sample. 
    # We need to create a mask tensor to avoid accounting for losses from the padding. 
    weight_mask = tf.to_float(tf.sequence_mask(seq_length))
    loss = seq2seq.sequence_loss(???)
    
    train_op = layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=params["learning_rate"],
        optimizer=tf.train.AdamOptimizer,
        clip_gradients=params['grad_clip'],
        summaries=[
            "learning_rate",
            "loss",
            "gradients",
            "gradient_norm",
        ])
    
    # Determine the predicted classes from logits
    pred_classes = tf.to_int32(tf.argmax(???))
    
    # Convert classes to whether each sequence is the beginning of a word (class = 0 or 3)
    pred_words = tf.logical_or(tf.equal(???), tf.equal(???))
    target_words = tf.logical_or(tf.equal(???), tf.equal(???))
    
    # Compute precisions and recall
    precision = metrics.streaming_precision(???)
    recall = metrics.streaming_recall(???)
    
    predictions = {
        "classes": pred_classes
    }
    eval_metric_ops = {
        "precision": precision,
        "recall": recall
    }
    
    return learn.ModelFnOps(mode, predictions, loss, train_op, eval_metric_ops=eval_metric_ops)

In [ ]:
if __name__ == "__main__":
    model_params = dict(num_class=len(CLASS_MAP), num_char=len(CHARS_MAP), emb_size=128, rnn_units=256,
                        input_keep_prob=0.85, output_keep_prob=0.85, learning_rate=10e-4, grad_clip=1.0, k=2)
    
    rnn_model = learn.Estimator(model_fn=???
                                , params=model_params
                                , model_dir="model/_rnn_model"
                                , config=learn.RunConfig(save_checkpoints_secs=30,
                                                         keep_checkpoint_max=2))

    train_input_fn = data_provider("data/_tf_records_k2/train", batch_size=128)
    test_input_fn = data_provider("data/_tf_records_k2/test", batch_size=512)

    validation_monitor = monitors.ValidationMonitor(input_fn=test_input_fn,
                                                    eval_steps=10,
                                                    every_n_steps=500,
                                                    name='validation')

    # rnn_model.fit(input_fn=train_input_fn, steps=1, monitors=[validation_monitor])
    # rnn_model.evaluate(input_fn=train_input_fn, steps=1)

    text = """นางกุหลาบขายกุหลาบจำหน่ายไม้ดอกไม้ประดับ"""
    tudkum(text, rnn_model, model_params['k'])